<a href="https://colab.research.google.com/github/sydcelsior/twsm2-hw/blob/main/Relation_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Modules

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.4 MB/s 
     |████████████████████████████████| 101 kB 11.9 MB/s 
     |████████████████████████████████| 596 kB 58.3 MB/s 
     |████████████████████████████████| 6.6 MB 45.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install pyvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyvis: filename=pyvis-0.2.1-py3-none-any.whl size=23688 sha256=9f74bf74b4d949cebafb810c8ac2445f58d56704c632abfd8730db54867e911c
  Stored in directory: /root/.cache/pip/wheels/2a/8f/04/6340d46afc74f59cc857a594ca1a2a14a1f4cbd4fd6c2e9306
Successfully built pyvis


In [3]:
!pip install pyyaml==5.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 636 kB 5.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


In [255]:
!pip install wikipedia-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.5.4-py3-none-any.whl size=13477 sha256=94142dbee8a013b011baeee07eea095d2e0b1841bbcdb8bbb4db2695516f64c0
  Stored in directory: /root/.cache/pip/wheels/d3/24/56/58ba93cf78be162451144e7a9889603f437976ef1ae7013d04
Successfully built wikipedia-api


In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import math
import torch
import pandas as pd
import numpy as np 
from pyvis.network import Network
import IPython
from transformers import pipeline
import collections 
from collections import defaultdict
import plotly
import plotly.express as px
import seaborn as sns
import re
import itertools

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [281]:
from transformers import LukeTokenizer, LukeForEntityPairClassification
import spacy
import nltk.data
import wikipediaapi
import nltk
nltk.download('punkt')
from nltk import word_tokenize,sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large-finetuned-tacred")
model = LukeForEntityPairClassification.from_pretrained("studio-ousia/luke-large-finetuned-tacred")
nlp = spacy.load("en_core_web_sm")
triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.6M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/33.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/123 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/344 [00:00<?, ?B/s]

In [7]:
pd. set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Functions and classes

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
hearst_patterns = [
            (
                '(NP_\\w+ (, )?such as (NP_\\w+ ?(, )?(and |or )?)+)',
                'first'
            ),
            (
                '(such NP_\\w+ (, )?as (NP_\\w+ ?(, )?(and |or )?)+)',
                'first'
            ),
            (
                '((NP_\\w+ ?(, )?)+(and |or )?other NP_\\w+)',
                'last'
            ),
            (
                '(NP_\\w+ (, )?include (NP_\\w+ ?(, )?(and |or )?)+)',
                'first'
            ),
            (
                '(NP_\\w+ (, )?especially (NP_\\w+ ?(, )?(and |or )?)+)',
                'first'
            ),
            (
                '((NP_\\w+ ?(, )?)+(and |or )?any other NP_\\w+)',
                'last'
            ),
            (
                '((NP_\\w+ ?(, )?)+(and |or )?some other NP_\\w+)',
                'last'
            ),
            (
                '((NP_\\w+ ?(, )?)+(and |or )?be a NP_\\w+)',
                'last'
            ),
            (
                '(NP_\\w+ (, )?like (NP_\\w+ ? (, )?(and |or )?)+)',
                'first'
            ),
            (
                'such (NP_\\w+ (, )?as (NP_\\w+ ? (, )?(and |or )?)+)',
                'first'
            ),
            (
                '((NP_\\w+ ?(, )?)+(and |or )?like other NP_\\w+)',
                'last'
            ),
            (
                '((NP_\\w+ ?(, )?)+(and |or )?one of the NP_\\w+)',
                'last'
            ),
            (
                '((NP_\\w+ ?(, )?)+(and |or )?one of these NP_\\w+)',
                'last'
            ),
            (
                '((NP_\\w+ ?(, )?)+(and |or )?one of those NP_\\w+)',
                'last'
            ),
            (
                'example of (NP_\\w+ (, )?be (NP_\\w+ ? '
                '(, )?(and |or )?)+)',
                'first'
            ),
            (
                '((NP_\\w+ ?(, )?)+(and |or )?be example of NP_\\w+)',
                'last'
            ),
            (
                '(NP_\\w+ (, )?for example (, )?'
                '(NP_\\w+ ?(, )?(and |or )?)+)',
                'first'
            ),
            (
                '((NP_\\w+ ?(, )?)+(and |or )?which be call NP_\\w+)',
                'last'
            ),
            (
                '((NP_\\w+ ?(, )?)+(and |or )?which be name NP_\\w+)',
                'last'
            ),
            (
                '(NP_\\w+ (, )?mainly (NP_\\w+ ? (, )?(and |or )?)+)',
                'first'
            ),
            (
                '(NP_\\w+ (, )?mostly (NP_\\w+ ? (, )?(and |or )?)+)',
                'first'
            ),
            (
                '(NP_\\w+ (, )?notably (NP_\\w+ ? (, )?(and |or )?)+)',
                'first'
            ),
            (
                '(NP_\\w+ (, )?particularly (NP_\\w+ ? '
                '(, )?(and |or )?)+)',
                'first'
            ),
            (
                '(NP_\\w+ (, )?principally (NP_\\w+ ? (, )?(and |or )?)+)',
                'first'
            ),
            (
                '(NP_\\w+ (, )?in particular (NP_\\w+ ? '
                '(, )?(and |or )?)+)',
                'first'
            ),
            (
                '(NP_\\w+ (, )?except (NP_\\w+ ? (, )?(and |or )?)+)',
                'first'
            ),
            (
                '(NP_\\w+ (, )?other than (NP_\\w+ ? (, )?(and |or )?)+)',
                'first'
            ),
            (
                '(NP_\\w+ (, )?e.g. (, )?(NP_\\w+ ? (, )?(and |or )?)+)',
                'first'
            ),
            (
                '(NP_\\w+ \\( (e.g.|i.e.) (, )?(NP_\\w+ ? (, )?(and |or )?)+'
                '(\\. )?\\))',
                'first'
            ),
            (
                '(NP_\\w+ (, )?i.e. (, )?(NP_\\w+ ? (, )?(and |or )?)+)',
                'first'
            ),
            (
                '((NP_\\w+ ?(, )?)+(and|or)? a kind of NP_\\w+)',
                'last'
            ),
            (
                '((NP_\\w+ ?(, )?)+(and|or)? kind of NP_\\w+)',
                'last'
            ),
            (
                '((NP_\\w+ ?(, )?)+(and|or)? form of NP_\\w+)',
                'last'
            ),
            (
                '((NP_\\w+ ?(, )?)+(and |or )?which look like NP_\\w+)',
                'last'
            ),
            (
                '((NP_\\w+ ?(, )?)+(and |or )?which sound like NP_\\w+)',
                'last'
            ),
            (
                '(NP_\\w+ (, )?which be similar to (NP_\\w+ ? '
                '(, )?(and |or )?)+)',
                'first'
            ),
            (
                '(NP_\\w+ (, )?example of this be (NP_\\w+ ? '
                '(, )?(and |or )?)+)',
                'first'
            ),
            (
                '(NP_\\w+ (, )?type (NP_\\w+ ? (, )?(and |or )?)+)',
                'first'
            ),
            (
                '((NP_\\w+ ?(, )?)+(and |or )? NP_\\w+ type)',
                'last'
            ),
            (
                '(NP_\\w+ (, )?whether (NP_\\w+ ? (, )?(and |or )?)+)',
                'first'
            ),
            (
                '(compare (NP_\\w+ ?(, )?)+(and |or )?with NP_\\w+)',
                'last'
            ),
            (
                '(NP_\\w+ (, )?compare to (NP_\\w+ ? (, )?(and |or )?)+)',
                'first'
            ),
            (
                '(NP_\\w+ (, )?among -PRON- (NP_\\w+ ? '
                '(, )?(and |or )?)+)',
                'first'
            ),
            (
                '((NP_\\w+ ?(, )?)+(and |or )?as NP_\\w+)',
                'last'
            ),
            (
                '(NP_\\w+ (, )? (NP_\\w+ ? (, )?(and |or )?)+ '
                'for instance)',
                'first'
            ),
            (
                '((NP_\\w+ ?(, )?)+(and|or)? sort of NP_\\w+)',
                'last'
            ),
            (
                '(NP_\\w+ (, )?which may include (NP_\\w+ '
                '?(, )?(and |or )?)+)',
                'first'
            )
        ]

In [230]:
class KB():
    def __init__(self):
        self.entities = {}
        self.relations = []
        self.path = r"/content/drive/MyDrive/twsm/isontology.csv"
        self.df = pd.read_csv(self.path, index_col=0)
        self.adjacency_graph = {}
        self.inverse_adjacency_graph = {}
        self.cand_dic = {}
        self.ontology = {}
        self.levels = {}
        self.counter = 0

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def add_relation(self, r):
        
        r["head"] = self.entity_linking(r["head"])
        #print(r["head"])

        r["tail"] = self.entity_linking(r["tail"])
        #print(r["tail"])

        if not self.exists_relation(r):
            self.relations.append(r)

            # adding to adjacency_graph
            if r["head"] not in self.adjacency_graph:
              self.adjacency_graph[r["head"]] = [r["tail"]]
            elif r["head"] in self.adjacency_graph and r["tail"] not in self.adjacency_graph[r["head"]]:
              self.adjacency_graph[r["head"]].append(r["tail"])
            else:
              None
            
            # adding to inverse adjacency_graph
            if r["tail"] not in self.inverse_adjacency_graph:
              self.inverse_adjacency_graph[r["tail"]] = [r["head"]]
            elif r["tail"] in self.inverse_adjacency_graph and r["head"] not in self.inverse_adjacency_graph[r["tail"]]:
              self.inverse_adjacency_graph[r["tail"]].append(r["head"])
            else:
              None
        
        candidate_entities = [r["head"], r["tail"]]

        # manage new entities
        for ent in candidate_entities:
            self.add_entity_relation(ent)
    
    def entity_linking(self, ent):
        if self.df[self.df["synonym"].isin([ent.lower()])].shape[0] > 0:
          #print(True)
          return self.df[self.df["synonym"] == ent.lower()]["entity_id"].values[0]
        else:
          #print(False)
          return ent.lower()
    
    def change_synonym_db(self, path):
        self.path = path

    def add_entity_relation(self, e):
        self.entities[e] = e
      
    def add_entity(self, e):
        ent = self.entity_linking(e)
        self.entities[ent] = ent
        self.adjacency_graph[ent] = []
        self.inverse_adjacency_graph[ent] = []
    
    def delete_relation(self, head, tail):
      '''Method used for deleting a specific relation from the relations list'''

      relations_to_delete = []

      for rel in self.relations:
        if (rel["head"] == head) & (rel["tail"] == tail):
          relations_to_delete.append(rel)
      
      print(f"We need to remove the following relations {relations_to_delete}")

      if relations_to_delete != []:
        for rel_del in relations_to_delete:
          self.relations.remove(rel_del)

    def get_cand_heads(self):
        '''Method used to extract the incoming and outgoing nodes of the ontology'''

        # setting an empty dic, for each entity, with incoming equal 0 
        nodes_inc = dict.fromkeys(self.entities.keys(), 0)
        nodes_out = dict.fromkeys(self.entities.keys(), 0)
        dd = defaultdict(list)
        #cand_dic = dict.fromkeys(self.entities.keys(), [0,0])

        # calculating outgoing nodes
        for node in self.entities.keys():
          if node in self.adjacency_graph.keys():
            #self.cand_dic[node][0] = len(self.adjacency_graph[node])
            #cand_nodes[node].insert(0, len(self.adjacency_graph[node]))
            nodes_out[node] = len(self.adjacency_graph[node])
          if node in self.inverse_adjacency_graph.keys():
            #self.cand_dic[node][1] = len(self.inverse_adjacency_graph[node])
            #cand_nodes[node].insert(1, len(self.inverse_adjacency_graph[node]))
            nodes_inc[node] = len(self.inverse_adjacency_graph[node])

        for d in (nodes_inc, nodes_out): # you can list as many input dicts as you want here
          for key, value in d.items():
            dd[key].append(value)

        #print(dd)
        self.cand_dic = dd
        #print(self.cand_dic)
        #self.cand_dic = cand_nodes

    def plot_candidates(self):
      
      # running the method for updating the candidates
      self.get_cand_heads()

      df = pd.DataFrame(self.cand_dic).T.rename(columns = {0:"Incoming nodes", 1:"Outgoing nodes"})

      fig = px.scatter(df, x="Outgoing nodes", y="Incoming nodes", hover_name=df.index)

      return fig.show()




    def initiate_ontology(self, ent):
      '''Function used to instantiate the ontology based on an entity chosen from a head'''

      # we check whether the head points outwards to a node
      try:
        # executed if true
        outward_nodes = self.adjacency_graph[ent]
        print(f"{ent} is pointing towards: {outward_nodes}")
        # we know that our head is pointing outwards, thus we delete all the relations
        self.adjacency_graph.pop(ent)
        # we then iterate over each node that we had a relationship to
        for point_node in outward_nodes:
          print(f"We are looking at {point_node}")
          # now we delete each outward pointing relation in the inverse graph
          self.inverse_adjacency_graph[point_node].remove(ent)
          #print(self.inverse_adjacency_graph[point_node])
          
          # in case it was only pointing to the head, we can delete it
          if self.inverse_adjacency_graph[point_node] == []:
            self.inverse_adjacency_graph.pop(point_node)
          
          # lastly we delete all the relations from our relations list
          self.delete_relation(ent, point_node)

      except:
        # executed if error
        print(f"{ent} does not have outgoing edge")
    

    def recursive_traversal(self, nodes_level, parent):
      '''Recursive Function that descends for each node to the subnodes and deletes all cycles and outward pointing dependencies'''

      # for each node at the same level, we detect three aspects:
      
      # illegal nodes 
      ill_nodes = nodes_level.copy()
      ill_nodes.append(parent)

      same_level_nodes = nodes_level.copy()
      

      # direct cycles with nodes on the same level
      for node in nodes_level:
        print(f"We are at {node}")


      # we check whether the node points outwards
        if node in self.adjacency_graph.keys():

          # this executes if the node is pointing outwards
          #print(adj_graph[node])

          # we calculate the nodes that point outwards to nodes on another level other than the parent
          nodes_out_lev = list(set(self.adjacency_graph[node]).difference(ill_nodes))
          print(f"{node} is facing {nodes_out_lev} not on the same level")

          # if the returned list is not empty, we have outward pointing nodes that are not on the level and delete all relations
          if nodes_out_lev != []:
            for node_out in nodes_out_lev:
              print(f"This node is outside the level: {node_out}")
              # we delete outwards pointing
              self.adjacency_graph[node].remove(node_out)

              # we delete incoming edge
              self.inverse_adjacency_graph[node_out].remove(node)
              if self.adjacency_graph[node] == []:
                self.adjacency_graph.pop(node)

              # in case the outward node didnt have any other nodes pointing towards it, we delete it
              if self.inverse_adjacency_graph[node_out] == []:
                self.inverse_adjacency_graph.pop(node_out)
              
              # lastly we delete all the relations from our relations list
              self.delete_relation(node, node_out)

          #  # we calculate the nodes that point to entities on the same level
          nodes_lev = list(set(self.adjacency_graph[node]) & set(same_level_nodes))
          print(f"These nodes are on the same level {nodes_lev}")

          # in case there exists a relationship to nodes on the same level
          if nodes_lev != []:
            # now there exist two options

            for int_node in nodes_lev:

              # option 1: a direct cycle with another node -> delete cycle
              try: 
                if node in self.adjacency_graph[int_node]:
                  print(f"Direct Cycle detected: {int_node}, {node}")
                  # we delete the cycle 
                  self.adjacency_graph[node].remove(int_node)
                  self.adjacency_graph[int_node].remove(node)
                  self.inverse_adjacency_graph[node].remove(int_node)
                  self.inverse_adjacency_graph[int_node].remove(node)

                  # we update the adjacency graph
                  if self.adjacency_graph[node] == []:
                    self.adjacency_graph.pop(node)
                  if self.adjacency_graph[int_node] == []:
                    self.adjacency_graph.pop(node)

                  # we update the inverse adjacency graph, in case there exist no nodes pointing to towards a node with a cycle
                  if self.inverse_adjacency_graph[node] == []:
                    self.inverse_adjacency_graph.pop(node)
                  if self.inverse_adjacency_graph[int_node] == []:
                    self.inverse_adjacency_graph.pop(int_node)
                  
                  # lastly we delete all the relations from our relations list, meaning in both directions
                  self.delete_relation(node, int_node)
                  self.delete_relation(int_node, node)
                  
                  
                # option 2: a relation outwards -> change hierarchy
              except:
                print(f"My new parent is {int_node}")
                print(f"I delete the relationship to {parent}")
                self.adjacency_graph[node].remove(int_node)
                self.inverse_adjacency_graph[int_node].remove(node)

                # we update adjacency and inverse adjacency graphs if necessary
                if self.adjacency_graph[node] == []:
                    self.adjacency_graph.pop(node)
                if self.inverse_adjacency_graph[node] == []:
                    self.inverse_adjacency_graph[node].pop
                
                # lastly we delete all the relations from our relations list
                self.delete_relation(node, node_out)

                

        # we recursively call the function again on all nodes that are pointing towards the nodes
      for nodes_rec in nodes_level:
          
        print(f"we look at {nodes_rec}")


        if nodes_rec in self.inverse_adjacency_graph.keys():
          # in case the node has any nodes pointing towards it 
           sub_nodes_rec = self.inverse_adjacency_graph[nodes_rec].copy()
           print(f"We are running recursively on {nodes_rec}, the nodes pointing towards {nodes_rec} are {self.inverse_adjacency_graph[nodes_rec]}")
           self.recursive_traversal(self.inverse_adjacency_graph[nodes_rec], nodes_rec)
          
        else:
          # if it doesnt we switch to the next element
          continue


      
    def create_ontology(self, head):
      '''Method that accepts the user's choice of the ontology head and creates the whole ontology onwards'''

      self.initiate_ontology(head)

      sub_nodes = self.inverse_adjacency_graph[head].copy()

      self.recursive_traversal(sub_nodes, head)

      self.levels = {}

      self.update_levels(head)


    def update_levels(self,head):
      '''Method that initiates the traversal for the levels'''
      
      self.counter = 1 

      # we create a counter for each level and add the head at level 1
      self.levels[self.counter] = [head]

      print(f"we added {head} at level 1")

      # we then run through all the nodes below the head recursively
      pointing_nodes = self.inverse_adjacency_graph[head]

      return self.levels_recursive(pointing_nodes, self.counter)



    def levels_recursive(self, pointing_nodes, counter):
      '''Method that runs down the graph recursivelly and adds the nodes to the level'''
      # we arrive with the nodes that point towards the node above, add them to the levels and run recursively again
      self.counter += 1
      for node in pointing_nodes:
        print(f"We look at node {node} and add it at level {self.counter}")
        try:
          self.levels[self.counter].append(node)
        except:
          self.levels[self.counter] = [node]
  
      for node in pointing_nodes:
        try:
          pointing_nodes_below = self.inverse_adjacency_graph[node]
          print(f"We run recursively on {self.inverse_adjacency_graph[node]}")
          return self.levels_recursive(pointing_nodes_below, self.counter)
        except:
          continue


    def print(self):
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")

In [11]:
def from_small_text_to_kb(text, triplet_extractor, kb ,verbose=False):

    # We need to use the tokenizer manually since we need special tokens.
    extracted_text = triplet_extractor.tokenizer.batch_decode([triplet_extractor(text, return_tensors=True, return_text=False)[0]["generated_token_ids"]])

    # create kb
    for sentence_pred in extracted_text:
        relations = extract_triplets(sentence_pred)
        for r in relations:
            #print(r)
            kb.add_relation(r)

    return kb

In [12]:
# Function to parse the generated text and extract the triplets
def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets

In [13]:
def sequential_tripplet_finder_luke(text, nlp_model, tokenizer_rel, model_rel, kb):
  '''A sequential way that separates the search for entities and relationships, based on spacy and luke'''


  doc = nlp_model(text)

  entities = [ent.text for ent in doc.ents]

  entity_spans = [(ent.start_char, ent.end_char) for ent in doc.ents]
  print(entity_spans)

  # no entities exist we can continue like that
  if len(entity_spans) == 0:
    return kb

  # if only one entity exist, we add it
  elif len(entity_spans) == 1:
    kb.add_entity(entities[0])
    return kb
  
  # if two exist, we check whether there exists a relationship
  elif len(entity_spans) == 2:
    inputs = tokenizer_rel(text, entity_spans=entity_spans, return_tensors="pt")
    outputs = model_rel(**inputs)
    logits = outputs.logits
    predicted_class_idx = logits.argmax(-1).item()
    if model_rel.config.id2label[predicted_class_idx] == "no_relation":
      kb.add_entity(entities[0])
      kb.add_entity(entities[1])
      return kb
    else:
      tripplet = {'head': entities[0], 'type': model_rel.config.id2label[predicted_class_idx], 'tail': entities[1]}
      kb.add_relation(tripplet)
      return kb
  
  # 
  elif len(entity_spans) > 2:

    combination_list = list(itertools.combinations(entity_spans, 2))
    ent_combination = list(itertools.combinations(entities, 2))
    print(combination_list)

    for index, pair in enumerate(combination_list):
      print(pair)
      inputs = tokenizer_rel(text, entity_spans=list(pair), return_tensors="pt")
      outputs = model_rel(**inputs)
      logits = outputs.logits
      predicted_class_idx = logits.argmax(-1).item()

      if model_rel.config.id2label[predicted_class_idx] == "no_relation":
        print("no relation found")
        kb.add_entity(ent_combination[index][0])
        kb.add_entity(ent_combination[index][1])
      else:
        tripplet = {'head': ent_combination[index][0], 'type': model_rel.config.id2label[predicted_class_idx], 'tail': ent_combination[index][1]}
        kb.add_relation(tripplet)
        print("adding tripplet")

    return kb

In [14]:
def sequential_tripplet_finder_hearst(text, nlp_model, hearst_patterns ,kb):
  '''A sequential way that separates the search for entities and relationships, based on spacy and hearst patterns'''

  # we run the entity recognizer and retrieve the entities and spans
  doc = nlp_model(text)

  entities = [ent.text for ent in doc.ents]

  entity_spans = [(ent.start_char, ent.end_char) for ent in doc.ents]
  print(entity_spans)

  # in case no entities are found we return the kb untouched
  if len(entity_spans) == 0:
    return kb

  # in case one is found, we add it to the knowledge base
  elif len(entity_spans) == 1:
    kb.add_entity(entities[0])
    return kb
  
  # in case two are found we have to evaluate
  elif len(entity_spans) == 2:
    result_hearst_search = hearst_pattern_search(entities, text, hearst_patterns)
    
    # noo relation, we  add the entities plain
    if result_hearst_search[0] == False:
      kb.add_entity(entities[0])
      kb.add_entity(entities[1])
      return kb

    # in case we find a relationship we determine the type of relatio, i.e. subgroup 
    elif result_hearst_search[0] == True and result_hearst_search[0] == "first":
      tripplet = {'head': entities[0], 'type': "subgroup of", 'tail': entities[1]}
      print(tripplet)
      kb.add_relation(tripplet)
      return kb
    else:
      tripplet = {'head': entities[1], 'type': "subgroup of", 'tail': entities[0]}
      print(tripplet)
      kb.add_relation(tripplet)
      return kb

  # in case we have more than two elements, we have to get all possible combinations
  elif len(entity_spans) > 2:
    combination_list = list(itertools.combinations(entities, 2))

    for pair in combination_list:

      entities = list(pair)

      result_hearst_search = hearst_pattern_search(entities, text, hearst_patterns)
    
      # noo relation, we  add the entities plain
      if result_hearst_search[0] == False:
        kb.add_entity(entities[0])
        kb.add_entity(entities[1])

      # in case we find a relationship we determine the type of relatio, i.e. subgroup 
      elif result_hearst_search[0] == True and result_hearst_search[0] == "first":
        tripplet = {'head': entities[0], 'type': "subgroup of", 'tail': entities[1]}
        print(tripplet)
        kb.add_relation(tripplet)

      else:
        tripplet = {'head': entities[1], 'type': "subgroup of", 'tail': entities[0]}
        print(tripplet)
        kb.add_relation(tripplet)
  
    return kb

In [15]:
def hearst_pattern_search(entity_list, sentence, hearst_patterns):
  '''Function for searching for relations using hearst patterns'''
  
  for ent in entity_list:
    sentence = re.sub(ent, "NP__", sentence)
  
  for pattern in hearst_patterns:
    match = re.search(pattern[0], sentence)
    try:
      match.group()
      return [True, pattern[1]]
      break
    except:
      continue

  return [False]

In [17]:
def save_network_html(kb, filename="network.html"):
  '''A function for creating a visual graph based on the kb and ntworkx'''
  # create network
  net = Network(directed=True, width="700px", height="700px", bgcolor="#eeeeee")

  palette = sns.color_palette(None, kb.counter).as_hex()

  if kb.levels == {}:
    size = 5

  # nodes
  for e in kb.entities:
      try:
        ent_lev = [level for level, ent_list in kb.levels.items() if e in ent_list][0]
        color_entity = palette[ent_lev - 1]
        size = ent_lev
        net.add_node(e, shape="circle", color=color_entity, size = size)
      except:
        color_entity = "#D5DBDB"
        net.add_node(e, shape="circle", color=color_entity, size = 5)

  # edges
  for r in kb.relations:
      net.add_edge(r["head"], r["tail"],
                  title=r["type"], label=r["type"])
      
  # save network
  net.repulsion(
      node_distance=200,
      central_gravity=0.2,
      spring_length=200,
      spring_strength=0.05,
      damping=0.09
  )
  net.set_edge_smooth('dynamic')
  net.show(filename)

  return net

In [309]:
def full_text_to_kb(text , kb, method = "rebel"):
  '''A function for converting a full text into a kb using either of the three methods'''

  sentences = sent_tokenize(text)

  for sentence in sentences:
    if method == "rebel":
      kb = from_small_text_to_kb(sentence, triplet_extractor, kb)
    elif method == "luke":
      kb = sequential_tripplet_finder_luke(sentence, nlp, tokenizer, model, kb)
    else:
      kb = kb_hearst = sequential_tripplet_finder_hearst(sentence, nlp, hearst_patterns ,kb_hearst)

  return kb

# Comparison of methods: Relation Extraction - REBEL

At first, let us compare an easy example using all three methods and the corresponding knowledge classes

## Method 1: Relation Classification - REBEL

In [114]:
# we instantiate an empty method
kb_rebel = KB()

In [115]:
# we use a sentence and feed it into the kb using rebel

text = "In Europe exist lot of countries such as Germany and France"
kb_rebel = from_small_text_to_kb(text, triplet_extractor, kb_rebel)

In [116]:
# we can evaluate the tagged relations and entities

print(kb_rebel.entities, kb_rebel.relations)


{'germany': 'germany', 'europe': 'europe', 'france': 'france'} [{'head': 'germany', 'type': 'continent', 'tail': 'europe'}, {'head': 'france', 'type': 'continent', 'tail': 'europe'}]


In [117]:
# we can display the graph visually 

filename = "network_test.html"
net = save_network_html(kb_rebel, filename=filename)
IPython.display.HTML(filename=filename)

## Method 2: Entity Recognition + Relation Extraction (Luke) 

In [22]:
# we instantiate the knowledge base again 

kb_luke = KB()

In [23]:
text = "In Europe exist lot of countries such as Germany and France"

kb_luke = sequential_tripplet_finder_luke(text, nlp, tokenizer, model, kb_luke)

[(3, 9), (41, 48), (53, 59)]
[((3, 9), (41, 48)), ((3, 9), (53, 59)), ((41, 48), (53, 59))]
((3, 9), (41, 48))
adding tripplet
((3, 9), (53, 59))
adding tripplet
((41, 48), (53, 59))
no relation found


In [24]:
# we can evaluate the tagged relations and entities

print(kb_luke.entities, kb_luke.relations)

{'europe': 'europe', 'germany': 'germany', 'france': 'france'} [{'head': 'europe', 'type': 'org:country_of_headquarters', 'tail': 'germany'}, {'head': 'europe', 'type': 'org:country_of_headquarters', 'tail': 'france'}]


In [25]:
# we can display the graph visually 

filename = "network_test.html"
net = save_network_html(kb_luke, filename=filename)
IPython.display.HTML(filename=filename)

## Method 3: Entity Recognition + Relation Extraction (Hearst Patterns) 

In [26]:
# we instantiate the knowledge base again 

kb_hearst = KB()

In [27]:
text = "In Europe exist lot of countries such as Germany and France"

kb_hearst = sequential_tripplet_finder_hearst(text, nlp, hearst_patterns ,kb_hearst)

[(3, 9), (41, 48), (53, 59)]


In [28]:
# we can evaluate the tagged relations and entities

print(kb_hearst.entities, kb_hearst.relations)

{'europe': 'europe', 'germany': 'germany', 'france': 'france'} []


In [29]:
# we can display the graph visually 

filename = "network_test.html"
net = save_network_html(kb_hearst, filename=filename)
IPython.display.HTML(filename=filename)

## Discussion

It seems as if the hardcoded way of finding relationships performs worse than the models that are trained specifically to find relations.

# More features of the knowledge base

Let's now look in more details at some of the features of the knowledge base.

### Entity Linking

In [30]:
kb = KB()
text = "Literature reviews are a qualitative method"
kb = from_small_text_to_kb(text, triplet_extractor, kb)

In [31]:
filename = "network_test.html"
net = save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

As we can evaluate the term literature reviews was transformed to the entity literature study, since it is linked to an entity given in the isontology.csv

### Adjacency Graphs and Candidates

We will create a different example containing of several sentences

In [231]:
kb = KB()
text = "Markus Löcher is a professer at the HWR Berlin"
kb = from_small_text_to_kb(text, triplet_extractor, kb)

In [232]:
text = "Roland Müller is the boss of Markus Löcher"
kb = from_small_text_to_kb(text, triplet_extractor, kb)

In [233]:
text = "Roland Müller is also a professor at HWR Berlin"
kb = from_small_text_to_kb(text, triplet_extractor, kb)

In [234]:
text = "Jonas Pieper is employed by Roland Müller"
kb = from_small_text_to_kb(text, triplet_extractor, kb)

In [235]:
filename = "network_test.html"
net = save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

What we can observe given this result is, that REBEL is capable to extract meaningful relationships between entities, even though it never saw them during training. Given this result, we might be interested to convert it into a hierarchical ontology. At first, we can get information through the adjacency graph (ent pointing to ent) and the inverse adjacency graph (ent being pointed at by ent).

In [236]:
kb.adjacency_graph

{'jonas pieper': ['roland müller'],
 'markus löcher': ['hwr berlin', 'roland müller'],
 'roland müller': ['hwr berlin']}

In [237]:
kb.inverse_adjacency_graph

{'hwr berlin': ['markus löcher', 'roland müller'],
 'roland müller': ['markus löcher', 'jonas pieper']}

We can have a more meaningful understanding for the likelihood of an entity being an important node, i.e. a node higher in the tree, by plotting the amount of incoming and outgoing nodes against each other.

In [238]:
kb.plot_candidates()

Even if the reader didn't have an understanding of the context of the nodes, we'd now be able to identify candidate nodes for the head of the ontology, i.e. the nodes on the top left -> hwr berlin <br>
Once we have a chosen candidate, we can start to recursively traverse the graph from the head and convert the ontology in a way that nodes can only point towards nodes that are higher above them.

In [239]:
kb.create_ontology("hwr berlin")

hwr berlin does not have outgoing edge
We are at markus löcher
markus löcher is facing [] not on the same level
These nodes are on the same level ['roland müller']
We are at roland müller
roland müller is facing [] not on the same level
These nodes are on the same level []
we look at markus löcher
we look at roland müller
We are running recursively on roland müller, the nodes pointing towards roland müller are ['markus löcher', 'jonas pieper']
We are at markus löcher
markus löcher is facing ['hwr berlin'] not on the same level
This node is outside the level: hwr berlin
We need to remove the following relations [{'head': 'markus löcher', 'type': 'employer', 'tail': 'hwr berlin'}]
These nodes are on the same level []
We are at jonas pieper
jonas pieper is facing [] not on the same level
These nodes are on the same level []
we look at markus löcher
we look at jonas pieper
we added hwr berlin at level 1
We look at node roland müller and add it at level 2
We run recursively on ['markus löch

In [240]:
filename = "network_test.html"
net = save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

We can see that now each node that is on the same level has the same colour. Even if we now added a node and created a new relationship, we just have to call the method again from the same head to convert it into the structure.

In [242]:
text = "Frank Habermann is also a professor at HWR Berlin"
kb = from_small_text_to_kb(text, triplet_extractor, kb)

In [243]:
filename = "network_test.html"
net = save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

As we can tell, Frank Habermann is not considered to be part of the ontology, thus we traverse the graph from the same head again.

In [244]:
kb.create_ontology("hwr berlin")

hwr berlin does not have outgoing edge
We are at roland müller
roland müller is facing [] not on the same level
These nodes are on the same level []
We are at frank habermann
frank habermann is facing [] not on the same level
These nodes are on the same level []
we look at roland müller
We are running recursively on roland müller, the nodes pointing towards roland müller are ['markus löcher', 'jonas pieper']
We are at markus löcher
markus löcher is facing [] not on the same level
These nodes are on the same level []
We are at jonas pieper
jonas pieper is facing [] not on the same level
These nodes are on the same level []
we look at markus löcher
we look at jonas pieper
we look at frank habermann
we added hwr berlin at level 1
We look at node roland müller and add it at level 2
We look at node frank habermann and add it at level 2
We run recursively on ['markus löcher', 'jonas pieper']
We look at node markus löcher and add it at level 3
We look at node jonas pieper and add it at level 

In [245]:
filename = "network_test.html"
net = save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

We can extract the information on which level the node is via the levels attribute.

In [246]:
kb.levels

{1: ['hwr berlin'],
 2: ['roland müller', 'frank habermann'],
 3: ['markus löcher', 'jonas pieper']}

In case we now add a relationship that wasn't there between nodes on the same level, the graph traversal algorithm will again make sure that the ontological tree structure remains.

In [247]:
text = "Frank Habermann is a good colleague of Roland Müller"
kb = from_small_text_to_kb(text, triplet_extractor, kb)

In [248]:
filename = "network_test.html"
net = save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

In [249]:
kb.create_ontology("hwr berlin")

hwr berlin does not have outgoing edge
We are at roland müller
roland müller is facing [] not on the same level
These nodes are on the same level ['frank habermann']
Direct Cycle detected: frank habermann, roland müller
We need to remove the following relations [{'head': 'roland müller', 'type': 'student', 'tail': 'frank habermann'}]
We need to remove the following relations [{'head': 'frank habermann', 'type': 'student of', 'tail': 'roland müller'}]
We are at frank habermann
frank habermann is facing [] not on the same level
These nodes are on the same level []
we look at roland müller
We are running recursively on roland müller, the nodes pointing towards roland müller are ['markus löcher', 'jonas pieper']
We are at markus löcher
markus löcher is facing [] not on the same level
These nodes are on the same level []
We are at jonas pieper
jonas pieper is facing [] not on the same level
These nodes are on the same level []
we look at markus löcher
we look at jonas pieper
we look at fran

In [250]:
filename = "network_test.html"
net = save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

## Evaluating the performance on longer texts 

The different methods all work on short sentences, whenever a long text is parsed into the model, we first need to break it down into smaller sentences and run the chosen method on each sentence. As an example, we can use the wikipedia api to extract the summary of an article and parse each sentence into the different methods and compare the result once we initiate the ontology to start at the same head.

In [299]:
wiki_wiki = wikipediaapi.Wikipedia('en')

page_py = wiki_wiki.page('machine_learning')

In [300]:
page_py = wiki_wiki.page('machine_learning')
print("Page - Exists: %s" % page_py.exists())

Page - Exists: True


In [301]:
ml_summary = page_py.summary
print(ml_summary)

Machine learning (ML) is a field of inquiry devoted to understanding and building methods that 'learn', that is, methods that leverage data to improve performance on some set of tasks. It is seen as a part of artificial intelligence. Machine learning algorithms build a model based on sample data, known as training data, in order to make predictions or decisions without being explicitly programmed to do so. Machine learning algorithms are used in a wide variety of applications, such as in medicine, email filtering, speech recognition, and computer vision, where it is difficult or unfeasible to develop conventional algorithms to perform the needed tasks.A subset of machine learning is closely related to computational statistics, which focuses on making predictions using computers, but not all machine learning is statistical learning. The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning. Data mining is a related field of 

In [302]:
# we instantiate an empty ontology 

kb = KB()

In [304]:
kb = full_text_to_kb(ml_summary, kb)

In [305]:
kb.plot_candidates()

In [307]:
kb.create_ontology("machine learning")

machine learning is pointing towards: ['data', 'model', 'computational statistics', 'machine learning']
We are looking at data
We need to remove the following relations [{'head': 'machine learning', 'type': 'studies', 'tail': 'data'}]
We are looking at model
We need to remove the following relations [{'head': 'machine learning', 'type': 'studies', 'tail': 'model'}]
We are looking at computational statistics
We need to remove the following relations [{'head': 'machine learning', 'type': 'different from', 'tail': 'computational statistics'}]
We are looking at machine learning
We need to remove the following relations [{'head': 'machine learning', 'type': 'subclass of', 'tail': 'machine learning'}]
We are at data
data is facing [] not on the same level
These nodes are on the same level []
We are at model
model is facing [] not on the same level
These nodes are on the same level []
We are at computational statistics
computational statistics is facing [] not on the same level
These nodes ar

In [308]:
filename = "network_test.html"
net = save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

### Saving an ontology

Once we have an ontology we are satisfied with, we can save the graph as an html document and load it later. We only need to specify the path for saving and loading it.

In [310]:
net.save_graph("/content/drive/MyDrive/twsm/ml_graph.html")

In [311]:
graph_path = "/content/drive/MyDrive/twsm/ml_graph.html"
IPython.display.HTML(filename=graph_path)


# After this only experimental and debugging

In [ ]:
# choosing a text

#text = "A ontology is a visual representation of entities taken from a text."
text = "Markus Löcher is a professer at the HWR Berlin"
#text = "MongoDB is a form of NoSQL"
#text = "NoSQL is also a database"
#text = "There exist many different database provider, such as Oracle, SAP or PostgreSQL."
#text = "Deep Learning is used for classification"
#text = "Literature reviews are a qualitative method"
#text = "Machine learning is a field of inquiry devoted to understanding and building methods that 'learn', that is, methods that leverage data to improve performance on some set of tasks. It is seen as a part of artificial intelligence."
#text = "In the field of management, strategic management involves the formulation and implementation of the major goals and initiatives taken by an organization's managers on behalf of stakeholders, based on consideration of resources and an assessment of the internal and external environments in which the organization operates."


In [ ]:
# we instantiate our model
kb_bert = KB()

In [ ]:
# running the model on our text 

kb = from_small_text_to_kb(text, triplet_extractor, kb_bert)

In [ ]:
text = "Classification is done by machine learning"
#text = "Machine learning algorithms build a model based on sample data, known as training data, in order to make predictions or decisions without being explicitly programmed to do so."
#text = "Strategic management provides overall direction to an enterprise and involves specifying the organization's objectives, developing policies and plans to achieve those objectives, and then allocating resources to implement the plans."
kb = from_small_text_to_kb(text, triplet_extractor, kb_bert)

In [ ]:
#text = "Random forests are a type of ensemble method"
#text = "Machine learning algorithms are used in a wide variety of applications, such as in medicine, email filtering, speech recognition, and computer vision, where it is difficult or unfeasible to develop conventional algorithms to perform the needed tasks."
#text = "Academics and practicing managers have developed numerous models and frameworks to assist in strategic decision-making in the context of complex environments and competitive dynamics."
#kb = from_small_text_to_kb(text, triplet_extractor, kb_bert)

In [ ]:
#text = "Strategic Management is a form of Management"
#kb = from_small_text_to_kb(text, triplet_extractor, kb_bert)

In [ ]:
text = "Roland Müller is the boss of Markus Löcher"
kb = from_small_text_to_kb(text, triplet_extractor, kb_bert)

In [ ]:
text = "Roland Müller is also a professor at HWR Berlin"
kb = from_small_text_to_kb(text, triplet_extractor, kb_bert)

In [ ]:
text = "Jonas Pieper is employed by Roland Müller"

In [ ]:
kb = from_small_text_to_kb(text, triplet_extractor, kb_bert)

In [ ]:
filename = "network_test.html"
net = save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

In [ ]:
kb.plot_candidates()

In [ ]:
kb.create_ontology("hwr berlin")

hwr berlin does not have outgoing edge
We are at markus löcher
markus löcher is facing [] not on the same level
These nodes are on the same level ['roland müller']
We are at roland müller
roland müller is facing [] not on the same level
These nodes are on the same level []
we look at markus löcher
we look at roland müller
We are running recursively on roland müller, the nodes pointing towards roland müller are ['markus löcher', 'jonas pieper']
We are at markus löcher
markus löcher is facing ['hwr berlin'] not on the same level
This node is outside the level: hwr berlin
We need to remove the following relations [{'head': 'markus löcher', 'type': 'employer', 'tail': 'hwr berlin'}]
These nodes are on the same level []
We are at jonas pieper
jonas pieper is facing [] not on the same level
These nodes are on the same level []
we look at markus löcher
we look at jonas pieper


In [ ]:
filename = "network_test.html"
net = save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

In [ ]:
text = "Markus Schaal is also a professor at HWR Berlin"
kb = from_small_text_to_kb(text, triplet_extractor, kb_bert)

In [ ]:
kb.levels

{1: ['hwr berlin'], 2: ['roland müller'], 3: ['markus löcher', 'jonas pieper']}

In [ ]:
kb.add_entity("Markus Schaal")

In [ ]:
kb.get_cand_heads()

In [ ]:
kb.create_ontology("hwr berlin")

hwr berlin does not have outgoing edge
We are at roland müller
We are at markus schaal
we look at roland müller
We are running recursively on roland müller, the nodes pointing towards roland müller are ['markus löcher', 'jonas pieper']
We are at markus löcher
markus löcher is facing [] not on the same level
These nodes are on the same level []
We are at jonas pieper
jonas pieper is facing [] not on the same level
These nodes are on the same level []
we look at markus löcher
we look at jonas pieper
we look at markus schaal


In [ ]:
filename = "network_test.html"
net = save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

In [ ]:
kb.levels

{1: ['hwr berlin'],
 2: ['roland müller'],
 3: ['markus löcher', 'jonas pieper'],
 5: ['roland müller', 'markus schaal'],
 6: ['markus löcher', 'jonas pieper']}

In [ ]:
text = "Random forests are a subclass of ensemble methods"
kb = from_small_text_to_kb(text, triplet_extractor, kb_bert)

In [ ]:
filename = "network_test.html"
net = save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

In [ ]:
kb.relations

[{'head': 'random forest classification',
  'tail': 'machine learning',
  'type': 'part of'},
 {'head': 'ensemble method', 'tail': 'machine learning', 'type': 'part of'},
 {'head': 'regression', 'tail': 'machine learning', 'type': 'facet of'},
 {'head': 'classification', 'tail': 'machine learning', 'type': 'subclass of'},
 {'head': 'randomm forest',
  'tail': 'ensemble metthods',
  'type': 'subclass of'}]

In [ ]:
kb.create_ontology("machine learning")

machine learning does not have outgoing edge
We are at random forest classification
random forest classification is facing [] not on the same level
These nodes are on the same level ['ensemble method']
We are at ensemble method
ensemble method is facing [] not on the same level
These nodes are on the same level []
We are at regression
regression is facing [] not on the same level
These nodes are on the same level []
We are at classification
classification is facing [] not on the same level
These nodes are on the same level []
we look at random forest classification
we look at ensemble method
We are running recursively on ensemble method, the nodes pointing towards ensemble method are ['random forest classification']
We are at random forest classification
random forest classification is facing ['machine learning'] not on the same level
This node is outside the level: machine learning
We need to remove the following relations [{'head': 'random forest classification', 'type': 'part of', '

In [ ]:
kb.delete_relation("randomm forest", "ensemble metthods")

In [ ]:
kb.adjacency_graph

{'artificial intelligence': ['machine learning'],
 'chess program': ['board game'],
 'model': ['machine learning'],
 'speech recognition': ['machine learning']}

In [ ]:
kb.inverse_adjacency_graph

{'board game': ['chess program'],
 'machine learning': ['artificial intelligence',
  'model',
  'speech recognition']}

In [ ]:
kb.inverse_adjacency_graph

{'management': ['managers', 'strategic management'],
 'model': ['framework'],
 'strategic management': ['objectives', 'policies', 'plans']}

In [ ]:
kb.add_relation()

In [ ]:
kb.relations

[{'head': 'management', 'tail': 'managers', 'type': 'practiced by'},
 {'head': 'managers',
  'tail': 'management',
  'type': 'field of this occupation'},
 {'head': 'objectives', 'tail': 'strategic management', 'type': 'part of'},
 {'head': 'policies', 'tail': 'strategic management', 'type': 'part of'},
 {'head': 'plans', 'tail': 'strategic management', 'type': 'part of'},
 {'head': 'framework', 'tail': 'model', 'type': 'subclass of'},
 {'head': 'strategic management', 'tail': 'management', 'type': 'subclass of'}]

In [ ]:
filename = "network_test.html"
net = save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

In [ ]:
text = "Speech recognition is a subgroup of audio analysis"
kb = from_small_text_to_kb(text, triplet_extractor, kb_bert)

In [ ]:
filename = "network_test.html"
net = save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

In [ ]:
kb.create_ontology("board game")

board game does not have outgoing edge
We are at chess program
chess program is facing [] not on the same level
These nodes are on the same level []
We are at model
model is facing [] not on the same level
These nodes are on the same level []
we look at chess program
we look at model
We are running recursively on model, the nodes pointing towards model are ['machine learning']
We are at machine learning
machine learning is facing [] not on the same level
These nodes are on the same level []
we look at machine learning
We are running recursively on machine learning, the nodes pointing towards machine learning are ['artificial intelligence', 'speech recognition']
We are at artificial intelligence
artificial intelligence is facing [] not on the same level
These nodes are on the same level []
We are at speech recognition
speech recognition is facing ['audio analysis'] not on the same level
This node is outside the level: audio analysis
We need to remove the following relations [{'head': 's

In [ ]:
filename = "network_test.html"
net = save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

In [ ]:
relat_list = kb.relations

In [ ]:
def delete_relation(rel_list, head, tail):
  '''Function used for deleting a specific relation from the relations list'''

  relations_to_delete = []

  for rel in rel_list:
    if (rel["head"] == head) & (rel["tail"] == tail):
      relations_to_delete.append(rel)
  
  if relations_to_delete != []:
    for rel_del in relations_to_delete:
      rel_list.remove(rel_del)


In [ ]:
delete_relation(relat_list, "management", "managers")

In [ ]:
relat_list

[{'head': 'managers',
  'tail': 'management',
  'type': 'field of this occupation'},
 {'head': 'objectives', 'tail': 'strategic management', 'type': 'part of'},
 {'head': 'policies', 'tail': 'strategic management', 'type': 'part of'},
 {'head': 'plans', 'tail': 'strategic management', 'type': 'part of'},
 {'head': 'framework', 'tail': 'model', 'type': 'subclass of'},
 {'head': 'strategic management', 'tail': 'management', 'type': 'subclass of'}]

In [ ]:
kb.inverse_adjacency_graph

{'management': ['managers', 'strategic management'],
 'model': ['framework'],
 'strategic management': ['objectives', 'policies', 'plans']}

In [ ]:
adj_graph_test = kb.adjacency_graph.copy()

We point to 

In [ ]:
adj_graph_test

{'framework': ['model'],
 'management': ['managers'],
 'managers': ['management'],
 'objectives': ['strategic management'],
 'plans': ['strategic management'],
 'policies': ['strategic management'],
 'strategic management': ['management']}

In [ ]:
adj_graph_test_inv = kb.inverse_adjacency_graph.copy()

We are pointed at by

In [ ]:
adj_graph_test_inv

{'management': ['managers', 'strategic management'],
 'managers': ['management'],
 'model': ['framework'],
 'strategic management': ['objectives', 'policies', 'plans']}

In [ ]:
adj_graph_test_inv["management"]

['managers', 'strategic management']

In [ ]:
def create_ontology_head(adj_graph, inv_adj_graph, ent):
  '''Function used to instantiate the ontology'''

  # we check whether the head points outwards to a node
  try:
    # executed if true
    outward_nodes = adj_graph[ent]
    print(outward_nodes)
    # we know that our head is pointing outwards, thus we delete all the relations
    adj_graph.pop(ent)
    # we then iterate over each node that we had a relationship to
    for point_node in outward_nodes:
      print(point_node)
      # now we delete each outward pointing relation in the inverse graph
      inv_adj_graph[point_node].remove(ent)
      print(inv_adj_graph[point_node])
      
      # in case it was only pointing to the head, we can delete it
      if inv_adj_graph[point_node] == []:
        inv_adj_graph.pop(point_node)
      # now our head is cycle-free
  except:
    # executed if error
    print(f"{ent} does not have outgoing edge")





In [ ]:
def recursive_traversal(adj_graph, inv_adj_graph, nodes_level, parent):
  '''Recursive Function that descends for each node the subnodes and deletes all cycles and outward pointing dependencies'''

  # for each node at the same level, we detect three aspects:
  
  # illegal nodes 
  ill_nodes = nodes_level.copy()
  ill_nodes.append(parent)

  same_level_nodes = nodes_level.copy()
  

  # direct cycles with nodes on the same level
  for node in nodes_level:
    print(f"We are at {node}")

  # we check whether the node points outwards
    if node in adj_graph.keys():

      # this executes if the node is pointing outwards
      #print(adj_graph[node])

      # we calculate the nodes that point outwards to nodes on another level other than the parent
      nodes_out_lev = list(set(adj_graph[node]).difference(ill_nodes))
      print(f"{node} is facing {nodes_out_lev} not on the same level")

      # if the returned list is not empty, we have outward pointing nodes that are not on the level an delete all relations
      if nodes_out_lev != []:
        for node_out in nodes_out_lev:
          print(f"This node is outside the level: {node_out}")
          # we delete outwards pointing
          adj_graph[node].remove(node_out)

          # we delete incoming edge
          inv_adj_graph[node_out].remove(node)
          if adj_graph[node] == []:
            adj_graph.pop(node)

          # in case the outward node didnt have any other nodes pointing towards it, we delete it
          if inv_adj_graph[node_out] == []:
            inv_adj_graph.pop(node_out)

      #  # we calculate the nodes that point to entities on the same level
      nodes_lev = list(set(adj_graph[node]) & set(same_level_nodes))
      print(f"These nodes are on the same level {nodes_lev}")

      # in case there exists a relationship to nodes on the same level
      if nodes_lev != []:
        # now there exist two options

        for int_node in nodes_lev:

          # option 1: a direct cycle with another node -> delete cycle
          try: 
            if node in adj_graph[int_node]:
              print(f"Direct Cycle detected: {int_node}, {node}")
              # we delete the cycle 
              adj_graph[node].remove(int_node)
              adj_graph[int_node].remove(node)
              inv_adj_graph[node].remove(int_node)
              inv_adj_graph[int_node].remove(node)

              # we update the adjacency graph
              if adj_graph[node] == []:
                adj_graph.pop(node)
              if adj_graph[int_node] == []:
                adj_graph.pop(node)

              # we update the inverse adjacency graph, in case there exist no nodes pointing to towards a node with a cycle
              if inv_adj_graph[node] == []:
                inv_adj_graph.pop(node)
              if inv_adj_graph[int_node] == []:
                inv_adj_graph.pop(int_node)
              
              
            # option 2: a relation outwards -> change hierarchy
          except:
            print(f"My new parent is {int_node}")
            print(f"I delete the relationship to {parent}")
            adj_graph[node].remove(int_node)
            inv_adj_graph[int_node].remove(node)

            # we update adjacency and inverse adjacency graphs if necessary
            if adj_graph[node] == []:
                adj_graph.pop(node)
            if inv_adj_graph[node] == []:
                inv_adj_graph[node].pop
            

    # we recursively call the function again on all nodes that are pointing towards the nodes
    for nodes_rec in nodes_level:
      print(f"We now call the same func recursively on node {nodes_rec}")

      # if the node has nodes pointing towards it, we call the function again 
      try:
        print(f"We are running the recursive algorithm on {nodes_rec}")
        sub_nodes_rec = adj_graph_test_inv[nodes_rec].copy()
        return recursive_traversal(adj_graph_test, adj_graph_test_inv, sub_nodes_rec, nodes_rec)
      except:
        # if it doesnt we leave it as it is
        print(f"For node {nodes_rec} exists no node on which to perform a recursive traversal")
        continue
      
      

In [ ]:
create_ontology_head(adj_graph_test, adj_graph_test_inv, "management")

['managers']
managers
[]


In [ ]:
adj_graph_test

{'framework': ['model'],
 'managers': ['management'],
 'objectives': ['strategic management'],
 'plans': ['strategic management'],
 'policies': ['strategic management'],
 'strategic management': ['management']}

In [ ]:
adj_graph_test_inv

{'management': ['managers', 'strategic management'],
 'model': ['framework'],
 'strategic management': ['objectives', 'policies', 'plans']}

In [ ]:
sub_nodes = adj_graph_test_inv["management"].copy()

In [ ]:
sub_nodes

['managers', 'strategic management']

In [ ]:
recursive_traversal(adj_graph_test, adj_graph_test_inv, sub_nodes, "management")

We are at managers
managers is facing [] not on the same level
These nodes are on the same level []
We now call the same func recursively on node managers
We are running the recursive algorithm on managers
For node managers exists no node on which to perform a recursive traversal
We now call the same func recursively on node strategic management
We are running the recursive algorithm on strategic management
We are at objectives
objectives is facing [] not on the same level
These nodes are on the same level []
We now call the same func recursively on node objectives
We are running the recursive algorithm on objectives
For node objectives exists no node on which to perform a recursive traversal
We now call the same func recursively on node policies
We are running the recursive algorithm on policies
For node policies exists no node on which to perform a recursive traversal
We now call the same func recursively on node plans
We are running the recursive algorithm on plans
For node plans ex

In [ ]:
adj_graph_test

{'framework': ['model'],
 'managers': ['management'],
 'objectives': ['strategic management'],
 'plans': ['strategic management'],
 'policies': ['strategic management'],
 'strategic management': ['management']}

In [ ]:
adj_graph_test_inv

{'management': ['managers', 'strategic management'],
 'model': ['framework'],
 'strategic management': ['objectives', 'policies', 'plans']}

In [ ]:
filename = "network_test.html"
net = save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

In [ ]:
adj_graph_test

{'categorical data': ['classification'],
 'classification': ['machine learning'],
 'ensemble method': ['machine learning', 'random forest classification'],
 'random forest classification': ['ensemble method', 'machine learning'],
 'regression': ['machine learning']}

### Old Version

In [ ]:
def graph_traversal(adj_graph, inv_adj_graph, ent):
  
  try:
    # extract entities on the sublevel
    ent_level = inv_adj_graph[ent]

    print(inv_adj_graph[ent])

    # iterate over entities pointing to parent
    for sub_ent in ent_level:
      

      #print(inv_adj_graph[sub_ent])

      print(f"Currently looked at: {sub_ent}")
      try:
        print(f"{sub_ent} is pointed at by {inv_adj_graph[sub_ent]}")
        if ent in inv_adj_graph[sub_ent]:
          print("There exists a cycle")
      except:
        print(f"No node pointing at {sub_ent}")

      #print(list(set(adj_graph[sub_ent]) & set(ent_level)))

      #if len(list(set(adj_graph[sub_ent]) & set(ent_level))) > 0:
      #  print(True)
      #  print(adj_graph[sub_ent])

      # delete direct cycles to parent
      try:
        if ent in inv_adj_graph[sub_ent]:
          cyc_ents = adj_graph[ent]
          print(cyc_ents)
          # we pop the key such that no arrow downwards
          adj_graph.pop(ent)
          for cyc_ent in cyc_ents:
            # we pop each element such that 
            inv_adj_graph[cyc_ent].remove(ent)
            # if empty we can delete the whole
            if inv_adj_graph[cyc_ent] == []:
              inv_adj_graph.pop(cyc_ent)
      except:
          pass

      # delete direct cycles to neighbours at same level

      # is there a directed edge to a node on the same level
      if len(list(set(adj_graph[sub_ent]) & set(ent_level))) > 0:
        print("We have a dependency to a node on the same level")

        # in case there is a node pointing to one on the same level, find if any points back
        for node_same_lev in list(set(adj_graph[sub_ent]) & set(ent_level)):
          print(node_same_lev)
          try:
            # in case any of the nodes points back at the initial entity
            if sub_ent in adj_graph[node_same_lev]:

              print("Deleting the cycle on this level")
              # remove the direct relation from both sides
              adj_graph[sub_ent].remove(node_same_lev)
              if adj_graph[sub_ent] == []:
                
              adj_graph[node_same_lev].remove(sub_ent)
              inv_adj_graph[sub_ent].remove(node_same_lev)
              inv_adj_graph[node_same_lev].remove(sub_ent)
              if inv_adj_graph[node_same_lev] == []:
                inv_adj_graph.pop(node_same_lev)
              if inv_adj_graph[sub_ent] == []:
                inv_adj_graph.pop(sub_ent)
          except:
            pass
            

      # delete direct relations on same level and change hierarchy
      
      # is there a directed edge to a node on the same level after deleting direct cycles, we know a we must change the hierarchy
      if len(list(set(adj_graph[sub_ent]) & set(ent_level))) > 0:
        print("Correct path")
        # in case there is none pointing back, so only a one-way relationship exists, we take the fist parent as a new parent
        parent = list(set(adj_graph[sub_ent]) & set(ent_level))[0]
        print(f"Parent: {parent}")
        # we get the parent of the parent, which can only be one node
        par_parent = adj_graph[parent][0]
        print(f"parent of parent: {par_parent}")
        # we change the relationship and hierarchy
        adj_graph[sub_ent].remove(par_parent)
        inv_adj_graph[par_parent].remove(sub_ent)
      

    # applying the function again on all subnodes
    for sub_ent in ent_level:
        graph_traversal(adj_graph, inv_adj_graph, sub_ent)
  

  except Exception:
    try:
      print(f"For entity {ent} exists no further subgroup")
      print(f"{ent} is pointing to {adj_graph[ent]}")
      # for each node that the current node is pointing towards, check that none of the upper groups are pointing backwards
      for pointed_node in adj_graph[ent]:
        try:
          if ent in adj_graph[pointed_node]:
            print(f"Wrong hierarchy detected: {pointed_node}")
        except:
          continue
    except Exception:
      next
    
  


In [ ]:
adj_graph_test

{'categorical data': ['classification'],
 'classification': ['machine learning', 'categorical data'],
 'ensemble method': ['machine learning', 'random forest classification'],
 'machine learning': ['classification'],
 'random forest classification': ['ensemble method', 'machine learning'],
 'regression': ['machine learning']}

In [ ]:
adj_graph_test_inv

{'categorical data': ['classification'],
 'classification': ['machine learning', 'categorical data'],
 'ensemble method': ['random forest classification'],
 'machine learning': ['random forest classification',
  'ensemble method',
  'regression',
  'classification'],
 'random forest classification': ['ensemble method']}

In [ ]:
graph_traversal(adj_graph_test, adj_graph_test_inv, "machine learning")

['random forest classification', 'ensemble method', 'regression', 'classification']
Currently looked at: random forest classification
random forest classification is pointed at by ['ensemble method']
We have a dependency to a node on the same level
ensemble method
Deleting the cycle on this level
Currently looked at: ensemble method
No node pointing at ensemble method
Currently looked at: regression
No node pointing at regression
Currently looked at: classification
classification is pointed at by ['machine learning', 'categorical data']
There exists a cycle
['classification']
For entity random forest classification exists no further subgroup
random forest classification is pointing to ['machine learning']
For entity ensemble method exists no further subgroup
ensemble method is pointing to ['machine learning']
For entity regression exists no further subgroup
regression is pointing to ['machine learning']
['categorical data']
Currently looked at: categorical data
categorical data is poin

In [ ]:
adj_graph_test

{'categorical data': ['classification'],
 'ensemble method': ['machine learning'],
 'random forest classification': ['machine learning'],
 'regression': ['machine learning']}

In [ ]:
adj_graph_test_inv

{'categorical data': ['classification'],
 'machine learning': ['random forest classification',
  'ensemble method',
  'regression',
  'classification']}

In [ ]:
net = save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

Steps to do:

1. Delete direct links to parent:
- if parent in 

In [ ]:
kb.get_cand_heads()

defaultdict(<class 'list'>, {'random forest classification': [0, 2], 'ensemble method': [1, 1], 'machine learning': [4, 0], 'regression': [0, 1], 'classification': [0, 1]})


In [ ]:
filename = "network_test.html"
net = save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

In [ ]:
kb.adjacency_graph

{'artificial intelligence': ['machine learning'],
 'chess program': ['board game'],
 'model': ['machine learning'],
 'speech recognition': ['machine learning']}

In [ ]:
net.save_graph("/content/drive/MyDrive/twsm/net_graph.html")

In [ ]:
# we can now add new sentences with the same pipeline to the entology and plot it again 

text = "Classification is done by machine learning"
kb = from_small_text_to_kb(text, triplet_extractor, kb_bert)
filename = "network_test.html"
save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

In [ ]:
# And see how the knowledge base expands with more text 

text = "Deep learning is a form of machine learning"

kb = from_small_text_to_kb(text, triplet_extractor, kb_bert)
filename = "network_test.html"
save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

In [ ]:
kb.get_cand_head_inc(6)

[('machine learning', 5), ('ensemble method', 1), ('classification', 1), ('random forest classification', 0), ('regression', 0), ('deep learning', 0)]


In [ ]:
kb.relations

[{'head': 'deep learning', 'tail': 'classification', 'type': 'use'},
 {'head': 'classification', 'tail': 'machine learning', 'type': 'part of'},
 {'head': 'machine learning', 'tail': 'classification', 'type': 'has effect'},
 {'head': 'deep learning', 'tail': 'machine learning', 'type': 'subclass of'}]

Idea 1.:
- evaluate the whole list of entities and spot cycles
- a direct cycle exists when ent_a is head of ent_b but there exists another relation where ent_b is head of ent_a

In [ ]:
# However, there is still room for improvement

text = "Mongo DB uses a document view"

kb = from_small_text_to_kb(text, triplet_extractor, kb_bert)
filename = "network_test.html"
save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

{'head': 'Mongo DB', 'type': 'uses', 'tail': 'document view'}


#### Evaluating the pipeline with the existing ontology

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sqlite3
import pandas as pd

In [ ]:
path = r"/content/drive/MyDrive/twsm/isrecon.db"
con = sqlite3.connect(path)
cur = con.cursor()
df = pd.read_sql_query("SELECT * from entities", con)
con.close

<function Connection.close>

In [ ]:
path_sent = r"/content/drive/MyDrive/twsm/sentences.csv"
sent_df = pd.read_csv(path_sent)

In [ ]:
df_merged = pd.merge(sent_df, df, on="sentence_id", how = "inner")

In [ ]:
df_merged.head()

,article_id_x,sentence_id,para_id_x,sentence,section_nr,last_section_title,last_subsection_title,sentence_type,references,number_of_citations,article_id_y,para_id_y,sentence_start,section_title,subsection_title,label,ent_id,level_1,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9,level_10,tool,datatype,paired,setting,related,groups,double,dsr,temporal,level,stage,type,role,assumption,attr_tool,attr_datatype,attr_paired,attr_setting,attr_related,attr_groups,attr_double,attr_dsr,attr_temporal,attr_level,attr_stage,attr_type,attr_role,attr_assumption
0,1,1_322_354,1_3,"OUC have the potential to become a real-time, constantly updated product knowledge repository to serve existing product support START_CITE (Nambisan, 2002) END_CITE CITE_b42 .",1,Introduction,NaN,PARAGRAPH,['1_b42'],1,1,1_3,322,Introduction,None,TOPIC,knowledge repository,domain specific entity,IS topic,knowledge management,knowledge repository,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1_467_502,1_3,"SAP, for example, has implemented its OUC (SAP Community Network) as part of its platform strategy (see START_CITE Huang et al., 2018) END_CITE CITE_b30 .",1,Introduction,NaN,PARAGRAPH,['1_b30'],1,1,1_3,467,Introduction,None,TOPIC,digital platform,domain specific entity,IS topic,digital economy,digital business,digital platform,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1_502_527,1_3,"Examples of other well-known OUC include Salesforce.com's Trailblazer community, BMC's Products community and Cisco's Technology & Support community.",1,Introduction,NaN,PARAGRAPH,[],0,1,1_3,502,Introduction,None,TECHNOLOGY,Salesforce.com,domain specific entity,IS technology,enterprise information system,operational information system,enterprise resource planning,ERP software package,Salesforce.com,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1_530_551,1_4,A key issue facing firms that seek to capitalize on the strategic value of OUC is ensuring active user participation.,1,Introduction,NaN,PARAGRAPH,[],0,1,1_4,530,Introduction,None,TOPIC,participatory design,domain specific entity,IS topic,systems development,system development method,human centered design,participatory design,None,None,None,None,None,None,None,None,design method,None,None,None,None,None,None,None,None,None,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,1,1_861_886,1_5,Bringing the two groups together may create complimentary knowledge or may synthesize knowledge in new ways that help solve problems and generate new ideas.,1,Introduction,NaN,PARAGRAPH,[],0,1,1_5,861,Introduction,None,PARTICIPANTS,group participant,domain specific entity,study object,participant,group participant,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,group,None,None,None,None,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
df_art1 = df_merged[df_merged["article_id_x"] == 1]

In [ ]:
df_art1 = df_art1[["sentence", "ent_id"]]

In [ ]:
df_art1.shape

(132, 2)

In [ ]:
df_art1.head()

,sentence,ent_id
0,"OUC have the potential to become a real-time, constantly updated product knowledge repository to serve existing product support START_CITE (Nambisan, 2002) END_CITE CITE_b42 .",knowledge repository
1,"SAP, for example, has implemented its OUC (SAP Community Network) as part of its platform strategy (see START_CITE Huang et al., 2018) END_CITE CITE_b30 .",digital platform
2,"Examples of other well-known OUC include Salesforce.com's Trailblazer community, BMC's Products community and Cisco's Technology & Support community.",Salesforce.com
3,A key issue facing firms that seek to capitalize on the strategic value of OUC is ensuring active user participation.,participatory design
4,Bringing the two groups together may create complimentary knowledge or may synthesize knowledge in new ways that help solve problems and generate new ideas.,group participant


In [ ]:
kb = KB()

In [ ]:
df_art1["sent_tr"] = df_art1.apply(lambda x: from_small_text_to_kb(x["sentence"], triplet_extractor, kb), axis = 1)

In [ ]:
filename = "network_test.html"
net = save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)
net.save_graph("/content/drive/MyDrive/twsm/net_graph.html")

In [ ]:
IPython.display.HTML(filename=filename)

In [ ]:
text = "Classification is done by machine learning"
kb = from_small_text_to_kb(text, triplet_extractor, kb_bert)

In [ ]:
df_sub = df_merged[(df_merged["ent_id"] == "digital_platform") | (df_merged["ent_id"] == "digital business")]

In [ ]:
df_sub.head()

In [ ]:
text = "As a result, it is expected that IoT will have a great impact on the economy by transforming many enterprises into digital businesses and facilitating new business models as well as improving efficiency and generating new forms of revenue"

In [ ]:
triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')

In [ ]:
triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')

filename = "network_test.html"

kb = from_small_text_to_kb(text, triplet_extractor)

save_network_html(kb, filename=filename)
IPython.display.HTML(filename=filename)

In [ ]:
filename = "/content/drive/MyDrive/twsm/net_graph.html"
IPython.display.HTML(filename=filename)

- extend KB class for adding new sentences
- extend pipeline to deal with bigger articles
- cross-validate entities with existing ontology or dic
- train the model to recognize new entities